<a href="https://colab.research.google.com/github/becauseimyoung/python/blob/master/%E2%80%9CUntitled0_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 24 09:48:06 2018
@author: Leon
内容：
以LeNet-5网络为模型，搭建CNN，对mnist手写数字进行训练及测试
给关键tensor添加name，方便保存模型后的调用
"""
 
import tensorflow.examples.tutorials.mnist.input_data as input_data
import tensorflow as tf
import time
tf.reset_default_graph()#如果保存模型时报错，应清除已有图
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
 
x = tf.placeholder(tf.float32,[None,784],name='x')
y = tf.placeholder(tf.float32,[None,10],name='y')
 
def weight_variable(shape,name):  
    initial = tf.truncated_normal(shape, stddev = 0.1,name=name) # 截断正态分布  
    return tf.Variable(initial)  
  
def bias_variable(shape,name):  
    initial = tf.constant(0.1, shape=shape, name=name) # 常量0.1  
    return tf.Variable(initial)
 
# =============================================================================
#  第一层：卷积+激活+池化
# =============================================================================
# 卷积核5*5,通道为1，个数为32个
filter1 = weight_variable([5,5,1,32],name='filter1')
# 卷积层：步长1*1,
# Padding 有两个参数：
# SAME——输出大小为:输入大小/s；
# VALID——输出大小为：(输入大小-f+1)/s
x_img = tf.reshape(x,[-1,28,28,1],name='x_img')
conv1 = tf.nn.conv2d(x_img, filter1, strides=[1,1,1,1], padding='SAME',name='conv1')
# 激活层：加偏移，然后激活
bias1 = bias_variable([32],name='bias1')
relu1 = tf.nn.relu(conv1+bias1,name='relu1')
# 池化层：窗口大小2*2，步长2*2
pool1 = tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='pool1')
print("第一层输出尺寸：",pool1.shape)
 
# =============================================================================
# 第二层：卷积+激活+池化
# =============================================================================
# 卷积核大小5*5,1通道，64个卷积核
filter2 = weight_variable([5,5,32,64],name='filter2')
# 卷积层：步长1*1
conv2 = tf.nn.conv2d(pool1,filter2,strides=[1,1,1,1],padding='SAME',name='conv2')
# 激活层：加偏移，然后激活
bias2 = bias_variable([64],name='bias2')
relu2 = tf.nn.relu(conv2+bias2,name='relu2')
# 池化层：
pool2 = tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='pool2')
print("第二层输出尺寸：", pool2.shape)
 
# =============================================================================
# 第三层：全连接层
# =============================================================================
# 提取上一层输出的尺寸
shape = pool2.shape.as_list()
# 拉伸上层输出，构造本层输入
fc_input = tf.reshape(pool2,[-1,shape[1]*shape[2]*shape[3]])
# 全连接层的权重：大小为本层输入*本层神经元的个数，也即下面的：[shape[1]*shape[2]*shape[3],32]
fc_w = weight_variable([shape[1]*shape[2]*shape[3],1024], name='fc_w')
# 全连接层的偏移量：大小为本层神经元的个数——32
fc_b = bias_variable([1024],name='fc_b')
# 建立本层全连接结构，并激活
fc_out = tf.nn.relu(tf.matmul(fc_input,fc_w)+fc_b,name='fc_out')
print("第三层输出尺寸：", fc_out.shape)
 
# Dropout，用来防止过拟合  
# 加在输出层之前，训练过程中开启dropout，测试过程中关闭  
keep_prob = tf.placeholder(tf.float32,name='keep_prob')
fc_out_drop = tf.nn.dropout(fc_out, keep_prob,name='fc_out_drop')
 
# =============================================================================
# 输出层
# =============================================================================
out_w = weight_variable([1024,10],name='out_w')
out_b = bias_variable([10],name='out_b')
pred = tf.nn.softmax(tf.matmul(fc_out_drop,out_w)+out_b,name='pred')
 
# 定义损失函数
loss = -tf.reduce_sum(y*tf.log(pred),name='loss')
# 定义优化器：Adam优化函数
optimizer = tf.train.AdamOptimizer(learning_rate=0.001,name='optimizer').minimize(loss)
 
# 定义评价指标——准确率
bool_pred = tf.equal(tf.arg_max(y,1), tf.arg_max(pred,1))
accuracy = tf.reduce_mean(tf.cast(bool_pred,tf.float32),name='accuracy')
 
# 定义全局变量初始化
init = tf.global_variables_initializer()
 
# 定义保存模型类
saver = tf.train.Saver()
 
# 启动图
with tf.Session() as sess:
    # 执行是初始化
    sess.run(init)
    # 训练样本中，总的batch个数
    total_batch_nums = int(mnist.train.num_examples/50)
    for epoch in range(1):
        avg_loss = 0.
        time_start = time.clock()
        for i in range(total_batch_nums):
            
            batch = mnist.train.next_batch(50)
            batch_xs,batch_ys = batch
#            batch_xs = batch_xs.reshape([-1,28,28,1])
            sess.run(optimizer,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.5})
#            avg_loss += sess.run(loss, feed_dict={x:batch_xs,y:batch_ys})
        time_end = time.clock()
        
        if epoch %1 == 0:
            accuracy_eval = sess.run(accuracy, 
                                     feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
            print("Epoch:","%04d"%(epoch+1),
#                  "loss:","%.4f"%avg_loss,
                  "accuracy:",accuracy_eval,
                  "running time:","%g"%(time_end-time_start))
            
    saver.save(sess,'model/my_model')
    accuracy_test = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
    print("test accuracy:",accuracy_test)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr